In [1]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from utils_training import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [ ]:
N_train = 1000
N_bound = 200

# x,t
la = np.array([1,1])
lb = np.array([-1,0])

traindata = lb+(la-lb)*lhs(2,N_train)

x_inside = traindata[:,0:1]
t_inside = traindata[:,1:2]

x_inside = numpy_to_tensor(x_inside, var_name="x_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
t_inside = numpy_to_tensor(t_inside, var_name="t_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

x_bound = lb[0]+(la[0]-lb[0])*lhs(1,N_bound)
t_bound = lb[1]+(la[1]-lb[1])*lhs(1,N_bound)

x_bound = numpy_to_tensor(x_bound, var_name="x_bound", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)
t_bound = numpy_to_tensor(t_bound, var_name="t_bound", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)

In [3]:
def exact_func(x_input):
    x_value = x_input[:, 0:1]
    t_value = x_input[:, 1:2]
    
    return  np.exp(-t_value) * np.sin(np.pi * x_value)

In [ ]:
traindata = np.concatenate((np.linspace(-1,1,200).reshape(-1,1),np.linspace(0,1,200).reshape(-1,1)),1)

x_plot = traindata[:,0:1]
t_plot = traindata[:,1:2]
xx,tt = np.meshgrid(x_plot,t_plot)

data_numpy = np.concatenate((xx.reshape(-1,1),tt.reshape(-1,1)),1)
aa = exact_func(data_numpy)

plt.imshow(aa.reshape(200,200),extent=[-1, 1, 0, 1], cmap='rainbow')
plt.colorbar(shrink=.5)

plt.show()

In [ ]:
random_seed = 1234
np.random.seed(random_seed)
observe_number = 10

observe_data = lb+(la-lb)*lhs(2,observe_number)
observe_clear_u = exact_func(observe_data)

############# N(0,0.1^2) #############
noise_nu = 0
noise_std = 0.1
noise_u = np.random.normal(loc=noise_nu, scale=noise_std, size=observe_clear_u.shape)
observe_u = observe_clear_u + noise_u
############# N(0,0.1^2) #############


observe_data = numpy_to_tensor(observe_data, var_name="observe_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
observe_clear_u = numpy_to_tensor(observe_clear_u, var_name="observe_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
observe_u = numpy_to_tensor(observe_u, var_name="observe_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
print('J:',len(observe_u))

observe_data_x_inside = observe_data[:,0:1]
observe_data_t_inside = observe_data[:,1:2]

noise_u = numpy_to_tensor(noise_u, var_name="noise_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)

In [ ]:
np.random.seed(5678)
N_test_number = 10000

test_data = lb+(la-lb)*lhs(2,N_test_number)
test_u = exact_func(test_data)

test_data = numpy_to_tensor(test_data, var_name="test_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
test_u = numpy_to_tensor(test_u, var_name="test_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

test_data_x_inside = test_data[:,0:1]
test_data_t_inside = test_data[:,1:2]

In [7]:
def output_transform(x_input, y_input):
    x_in = x_input[:, 0:1]
    t_in = x_input[:, 1:2]

    return (1 - x_in) * (1 + x_in) * (1 - torch.exp(-t_in)) * y_input + torch.sin(np.pi * x_in)

In [8]:
C1 = torch.tensor(2.0, requires_grad=True)

In [9]:
def get_loss_f(x_grad,t_grad,PINNs,C,return_sequence='not'):
    
    ########### loss f  ###########
    E_inside = PINNs(torch.cat((x_grad,t_grad),axis=1))
    E_inside = output_transform(torch.cat((x_grad,t_grad),axis=1),E_inside)
    
    u_xx = compute_higher_order_derivatives(E_inside, [x_grad,x_grad])
    u_t = compute_higher_order_derivatives(E_inside, [t_grad])
    
    loss_f_sequence = u_t-C*u_xx+torch.exp(-t_grad)*(torch.sin(torch.tensor(np.pi)*x_grad)-torch.tensor(np.pi)*torch.tensor(np.pi)*torch.sin(torch.tensor(np.pi)*x_grad))
    loss_f_squared_sequence = torch.square(loss_f_sequence)

    if return_sequence=='yes':
        return loss_f_squared_sequence
    else:
        return torch.mean(loss_f_squared_sequence)

In [10]:
def get_loss_bound(bound_x, bound_t, PINNs, C, return_sequence='not'):
    
    E_bound_x_zero = PINNs(torch.cat((bound_x,torch.zeros_like(bound_x)),axis=1)) 
    Exact_x_zero = torch.sin(torch.tensor(np.pi)*bound_x)
    loss_bound_for_a = torch.mean(torch.square(E_bound_x_zero-Exact_x_zero))
    
    E_bound_fu_1_t = PINNs(torch.cat((-torch.ones_like(bound_t),bound_t),axis=1)) 
    loss_bound_for_b = torch.mean(torch.square(E_bound_fu_1_t))
    
    E_bound_1_t = PINNs(torch.cat((torch.ones_like(bound_t),bound_t),axis=1))
    loss_bound_for_c = torch.mean(torch.square(E_bound_1_t))
    
    loss_bound_value = loss_bound_for_a+loss_bound_for_b+loss_bound_for_c
    
    return loss_bound_value

In [11]:
#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [12]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

net_settings_for_PINNs1 = NetSetting(input_dims=2, hidden_neurons_list=[20]*4, 
                                     output_dims=1, hidden_activation='tanh', 
                                     output_activation=None, initializer_method='xavier')
PINNs1 = get_mlp_pinn(net_settings_for_PINNs1)
PINNs1.cuda()

initialize_weights(PINNs1, net_settings_for_PINNs1.initializer_method)

optimizer1 = optim.Adam(PINNs1.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)    
optimizer1.add_param_group({'params': [C1], 'lr': 0.001})

In [13]:
x_inside_all = torch.cat((x_inside,observe_data[:,0:1]),axis=0)
t_inside_all = torch.cat((t_inside,observe_data[:,1:2]),axis=0)

In [ ]:
Theta_list = np.zeros_like(observe_u.cpu().detach().numpy())
Theta_list = numpy_to_tensor(Theta_list, var_name="Theta_list", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

In [15]:
C1_id = id(C1)

optimizer_error = optim.Adam([Theta_list], lr=0.0001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0.001, amsgrad=False)    

In [16]:
def train_optimizer1_error_correction(PINNs,C,Theta_list_p):
    
    for param in PINNs.parameters():
        param.requires_grad = True
    
    C.requires_grad = True
    
    Theta_list_p.requires_grad = False

In [17]:
def train_optimizer2_error_correction(PINNs,C,Theta_list_p):
    
    for param in PINNs.parameters():
        param.requires_grad = False
    
    C.requires_grad = False
    
    Theta_list_p.requires_grad = True

In [18]:
def train_for_error_correction(PINNs,C,Theta_list_input,observe_data_input,observe_u_input):
    
    train_optimizer2_error_correction(PINNs,C,Theta_list_input)
        
    E_observation = PINNs(observe_data_input)
    E_observation = output_transform(observe_data_input,E_observation)
    
    loss_observation_refine = torch.mean(torch.square(E_observation+Theta_list_input-observe_u_input)) 
    
    optimizer_error.zero_grad()
    loss_observation_refine.backward()
    optimizer_error.step()
        
    train_optimizer1_error_correction(PINNs,C,Theta_list_input)
    
    return Theta_list_input

In [19]:
def flatten_line(data):
    for i in range(1, len(data)):
        if data[i] > data[i - 1]:
            data[i] = data[i - 1]
    return data

In [20]:
import copy
def training_recall(PINNs, C, it_iter, loss_f_for_T_1_best, best_PINNs_state, 
                    loss_all_1, loss_f_1, loss_f_for_collocation_1, 
                    loss_f_for_T_1, loss_f_excapt_T_1, loss_T_1, 
                    loss_T_clear_1, loss_T_1_test_data,loss_T_EC_1, test_loss_1, 
                    C1_list, optimizer1):
    
    threshold= -0.1
    # 1e-3
    
    # Smooth the loss curve
    loss_f_for_T_1_smooth = flatten_line(loss_f_for_T_1.copy())
    current_min_index = loss_f_for_T_1_smooth.index(min(loss_f_for_T_1_smooth))
    current_min = min(loss_f_for_T_1_smooth)
    
    # Update the best state if current loss is lower
    if best_PINNs_state is None:
        best_PINNs_state = {
            'model_state': copy.deepcopy(PINNs.state_dict()),
            'C_state': C.clone().detach(),
            'optimizer_state': copy.deepcopy(optimizer1.state_dict()),
            'it': it_iter,
            'min_loss': current_min, 'rollback_pending' : False, 'record_current_min':None
        }

    elif (not best_PINNs_state['rollback_pending']) and current_min < best_PINNs_state['min_loss']:
            best_PINNs_state = {
                'model_state': copy.deepcopy(PINNs.state_dict()),
                'C_state': C.clone().detach(),
                'optimizer_state': copy.deepcopy(optimizer1.state_dict()),
                'it': it_iter,
                'min_loss': current_min, 'rollback_pending' : False, 'record_current_min':None
            }
        
    # Check if stagnation is detected
    if (not best_PINNs_state['rollback_pending']) and (len(loss_f_for_T_1_smooth) - current_min_index > 300):
        # Start storing states without immediate rollback
        print('Stagnation detected at iteration:', it_iter)
        best_PINNs_state['record_current_min'] = current_min
        best_PINNs_state['rollback_pending'] = True
        
    if best_PINNs_state['rollback_pending']:
        # Update the stored state with the latest
        best_PINNs_state['model_state'] = copy.deepcopy(PINNs.state_dict())
        best_PINNs_state['C_state'] = C.clone().detach()
        best_PINNs_state['optimizer_state'] = copy.deepcopy(optimizer1.state_dict())
        best_PINNs_state['it'] = it_iter
        
        #print(best_PINNs_state['record_current_min'] - current_min)
        # Check if loss has decreased beyond the threshold
        if (best_PINNs_state['record_current_min'] - current_min) > threshold:
            
            # Perform rollback to the latest stored state
            print('Restoring best model from iteration:', best_PINNs_state['it'])
            PINNs.load_state_dict(best_PINNs_state['model_state'])
            C_value = best_PINNs_state['C_state']
            with torch.no_grad(): 
                C.copy_(C_value)  
            optimizer1.load_state_dict(best_PINNs_state['optimizer_state'])
            recall_it = it_iter  
            it_iter = best_PINNs_state['it']  
            # Truncate the recorded losses up to the rollback point
            loss_all_1 = loss_all_1[:it_iter+1]
            loss_f_1 = loss_f_1[:it_iter+1]
            loss_f_for_collocation_1 = loss_f_for_collocation_1[:it_iter+1]
            loss_f_for_T_1 = loss_f_for_T_1[:it_iter+1]
            loss_f_excapt_T_1 = loss_f_excapt_T_1[:it_iter+1]
            loss_T_1 = loss_T_1[:it_iter+1]
            loss_T_clear_1 = loss_T_clear_1[:it_iter+1]
            loss_T_1_test_data = loss_T_1_test_data[:it_iter+1]
            test_loss_1 = test_loss_1[:it_iter+1]
            C1_list = C1_list[:it_iter+1]
            loss_f_for_T_1_best = True
            # Reset the rollback flag and recorded minimum loss
    
    return (PINNs, C, it_iter, loss_f_for_T_1_best, best_PINNs_state, 
            loss_all_1, loss_f_1, loss_f_for_collocation_1, loss_f_for_T_1, 
            loss_f_excapt_T_1, loss_T_1, loss_T_clear_1, loss_T_1_test_data,loss_T_EC_1, 
            test_loss_1, C1_list,optimizer1)

In [ ]:
############## Record list ###############
loss_all_1 = []
loss_f_1 = []
loss_f_for_collocation_1 = []
loss_f_for_T_1 = []
loss_f_excapt_T_1 = []
loss_T_1 = []
loss_T_clear_1 = []
loss_T_1_test_data = []
loss_T_EC_1 = []
test_loss_1 = []
C1_list = []
############## Record list ###############

nIter1 = 20000
it = 0


# Initial placeholders for best model and iteration
loss_f_for_T_1_best = False
best_PINNs_state = None
best_it = None
recall_it = None
k_iterations = 100


while it<nIter1:
    
    #########loss f#########    
    loss_f = get_loss_f(x_inside_all,t_inside_all,PINNs1,C1,return_sequence='not')

    #########loss f  for collocation data#########
    loss_f_for_collocation = get_loss_f(x_inside,t_inside,PINNs1,C1,return_sequence='not')
    #########loss f  for observation data#########
    loss_f_for_T = get_loss_f(observe_data_x_inside,observe_data_t_inside,PINNs1,C1,return_sequence='not')
    #########loss f  excapt observation data#########
    loss_f_excapt_T = get_loss_f(test_data_x_inside,test_data_t_inside,PINNs1,C1,return_sequence='not')
    
    #########loss observation#########        
    E_observation = PINNs1(observe_data) 
    E_observation = output_transform(observe_data,E_observation)
    
    if loss_f_for_T_1_best:
        loss_observation = torch.mean(torch.square(E_observation+Theta_list-observe_u)) 
    else:
        loss_observation = torch.mean(torch.square(E_observation-observe_u)) 
    
    #########loss EC######### 
    loss_EC = torch.mean(torch.square(Theta_list-noise_u))
    
    #########loss T noisy observation#########  
    loss_observation_noisy = torch.mean(torch.square(E_observation-observe_u)) 
    
    #########loss T excapt observation#########        
    E_observation_excapt = PINNs1(test_data) 
    E_observation_excapt = output_transform(test_data,E_observation_excapt)
    loss_observation_excapt = torch.mean(torch.square(E_observation_excapt-test_u))      

    #########loss T clear observation#########        
    E_observation_clear = PINNs1(observe_data) 
    E_observation_clear = output_transform(observe_data,E_observation_clear)
    loss_observation_clear = torch.mean(torch.square(E_observation_clear-observe_clear_u))     
    
    #########loss PI#########
    loss = loss_f+10*loss_observation
    
    #########test_loss NRMSE#########
    pre_u = PINNs1(test_data)
    pre_u = output_transform(test_data,pre_u)
    test_loss = relative_l2_torch(pre_u,test_u)
    #########test_loss NRMSE#########
    
    #########Record#########
    loss_f_1.append(loss_f.item())
    loss_f_for_collocation_1.append(loss_f_for_collocation.item())
    loss_f_for_T_1.append(loss_f_for_T.item())
    loss_f_excapt_T_1.append(loss_f_excapt_T.item())
    C1_list.append(C1.item())   
    loss_T_1.append(loss_observation_noisy.item()) 
    loss_T_1_test_data.append(loss_observation_excapt.item()) 
    loss_T_clear_1.append(loss_observation_clear.item()) 
    loss_T_EC_1.append(loss_EC.item()) 
    test_loss_1.append(test_loss)
    #########Record#########
    
    if it % 1000 == 0:
        print('It:', it, 'train_loss:', loss.item(), 'test_loss:', test_loss)
        print(C1)
        
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    

    if  not loss_f_for_T_1_best:
        if it>4500:
            (PINNs1, C1, it, loss_f_for_T_1_best, best_PINNs_state, 
             loss_all_1, loss_f_1, loss_f_for_collocation_1, loss_f_for_T_1, 
             loss_f_excapt_T_1, loss_T_1, loss_T_clear_1, loss_T_1_test_data,loss_T_EC_1, 
             test_loss_1, C1_list, optimizer1) = training_recall(
                 PINNs1, C1, it, loss_f_for_T_1_best, best_PINNs_state, 
                 loss_all_1, loss_f_1, loss_f_for_collocation_1, 
                 loss_f_for_T_1, loss_f_excapt_T_1, loss_T_1, loss_T_clear_1, 
                 loss_T_1_test_data,loss_T_EC_1, test_loss_1, C1_list, optimizer1)
            
            if loss_f_for_T_1_best:
                Theta_list.requires_grad = True
                for param_group in optimizer1.param_groups:
                    if any(id(param) == C1_id for param in param_group['params']):
                        param_group['lr'] = 0.0001  
    else:
        ############################ Refine L_T ############################          
        Theta_list = train_for_error_correction(PINNs1,C1,Theta_list,observe_data,observe_u)
        ############################ Refine L_T ############################  

    it = it + 1   
    
print('Final:', 'train_loss:', loss.item(), 'test_loss:', test_loss)